# उत्पादनमा AutoGen एजेन्टहरू: अवलोकन र मूल्याङ्कन

यस ट्युटोरियलमा, हामी **[Autogen एजेन्टहरू](https://github.com/microsoft/autogen)** का आन्तरिक चरणहरू (ट्रेसहरू) **निगरानी गर्ने** र [Langfuse](https://langfuse.com) प्रयोग गरेर यसको प्रदर्शन **मूल्याङ्कन गर्ने** तरिका सिक्नेछौं।

यो मार्गदर्शनले उत्पादनमा एजेन्टहरू छिटो र भरपर्दो रूपमा ल्याउन टोलीहरूले प्रयोग गर्ने **अनलाइन** र **अफलाइन** मूल्याङ्कन मापदण्डहरू समेट्छ। 

**किन AI एजेन्ट मूल्याङ्कन महत्त्वपूर्ण छ:**
- कार्यहरू असफल हुँदा वा कमजोर नतिजा उत्पादन गर्दा समस्या समाधान गर्न
- वास्तविक समयमा लागत र प्रदर्शन निगरानी गर्न
- निरन्तर प्रतिक्रिया मार्फत भरपर्दो र सुरक्षितता सुधार गर्न


## चरण १: वातावरण चरहरू सेट गर्नुहोस्

Langfuse API कुञ्जीहरू प्राप्त गर्न [Langfuse Cloud](https://cloud.langfuse.com/) मा साइन अप गर्नुहोस् वा [Langfuse स्व-होस्टिंग](https://langfuse.com/self-hosting) प्रयोग गर्नुहोस्।

_**नोट:** स्व-होस्ट गर्नेहरूले [Terraform मोड्युलहरू](https://langfuse.com/self-hosting/azure) प्रयोग गरेर Azure मा Langfuse तैनाथ गर्न सक्छन्। वैकल्पिक रूपमा, तपाईं [Helm चार्ट](https://langfuse.com/self-hosting/kubernetes-helm) प्रयोग गरेर Kubernetes मा Langfuse तैनाथ गर्न सक्नुहुन्छ।_


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

पर्यावरण भेरिएबलहरू सेट भएपछि, अब हामी Langfuse क्लाइन्टलाई इनिसियलाइज गर्न सक्छौं। `get_client()` ले वातावरण भेरिएबलहरूमा प्रदान गरिएको प्रमाणपत्रहरू प्रयोग गरेर Langfuse क्लाइन्टलाई इनिसियलाइज गर्छ।


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## चरण २: OpenLit उपकरणको सुरुवात गर्नुहोस्

अब, हामी [OpenLit](https://github.com/openlit/openlit) उपकरणको सुरुवात गर्छौं। OpenLit स्वचालित रूपमा AutoGen अपरेसनहरू समात्छ र OpenTelemetry (OTel) स्प्यानहरू Langfuse मा निर्यात गर्छ।


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## चरण ३: आफ्नो एजेन्ट चलाउनुहोस्

अब हामी हाम्रो उपकरण परीक्षण गर्न बहु-चरणीय एजेन्ट सेटअप गर्छौं।


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### ट्रेस संरचना

Langfuse ले **ट्रेस** रेकर्ड गर्छ, जसमा **स्प्यानहरू** हुन्छन्, जसले तपाईंको एजेन्टको तर्कको प्रत्येक चरणलाई प्रतिनिधित्व गर्छ। यहाँ, ट्रेसले समग्र एजेन्टको रन र निम्न स्प्यानहरू समावेश गर्दछ:
- खाना योजना बनाउने एजेन्ट
- पोषण विशेषज्ञ एजेन्टहरू

तपाईंले यीलाई निरीक्षण गरेर ठ्याक्कै समय कहाँ खर्च भएको छ, कति टोकन प्रयोग गरिएको छ, आदि हेर्न सक्नुहुन्छ:

![Langfuse मा ट्रेस ट्री](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[ट्रेसको लिङ्क](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## अनलाइन मूल्याङ्कन

अनलाइन मूल्याङ्कन भनेको एजेन्टलाई प्रत्यक्ष, वास्तविक वातावरणमा मूल्याङ्कन गर्नु हो, अर्थात् उत्पादनमा वास्तविक प्रयोगको क्रममा। यसमा एजेन्टको प्रदर्शनलाई वास्तविक प्रयोगकर्तासँगको अन्तरक्रियामा निरन्तर अनुगमन र नतिजाहरूको विश्लेषण गर्ने प्रक्रिया समावेश हुन्छ।

### उत्पादनमा ट्र्याक गर्नुपर्ने सामान्य मेट्रिक्सहरू

1. **खर्चहरू** — उपकरणले टोकन प्रयोगलाई ट्र्याक गर्छ, जसलाई प्रति टोकन मूल्य तोकेर अनुमानित खर्चमा रूपान्तरण गर्न सकिन्छ।
2. **ढिलाइ (Latency)** — प्रत्येक चरण पूरा गर्न, वा सम्पूर्ण प्रक्रिया सम्पन्न गर्न लाग्ने समयलाई अनुगमन गर्नुहोस्।
3. **प्रयोगकर्ताको प्रतिक्रिया** — प्रयोगकर्ताहरूले प्रत्यक्ष प्रतिक्रिया (जस्तै, थम्ब्स अप/डाउन) दिन सक्छन्, जसले एजेन्टलाई परिमार्जन वा सुधार गर्न मद्दत गर्दछ।
4. **LLM-as-a-Judge** — तपाईंको एजेन्टको उत्पादनलाई लगभग वास्तविक समयमा मूल्याङ्कन गर्न छुट्टै LLM प्रयोग गर्नुहोस् (जस्तै, विषाक्तता वा शुद्धता जाँच गर्न)।

तल, यी मेट्रिक्सका उदाहरणहरू देखाइएका छन्।


#### 1. लागतहरू

तलको स्क्रिनशटले `gpt-4o-mini` कलहरूको प्रयोग देखाउँछ। यो महँगो चरणहरू हेर्न र तपाईंको एजेन्टलाई अनुकूलन गर्न उपयोगी छ।

![Costs](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[ट्रेसको लिंक](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### २. ढिलाइ

हामीले प्रत्येक चरण पूरा गर्न कति समय लाग्यो भनेर पनि हेर्न सक्छौं। तलको उदाहरणमा, सम्पूर्ण प्रक्रिया पूरा गर्न करिब ३ सेकेन्ड लाग्यो, जसलाई तपाईं चरण अनुसार विभाजन गर्न सक्नुहुन्छ। यसले तपाईंलाई अवरोधहरू पहिचान गर्न र तपाईंको एजेन्टलाई अनुकूलन गर्न मद्दत गर्दछ।

![ढिलाइ](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png)

_[ट्रेसको लिङ्क](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### ३. प्रयोगकर्ता प्रतिक्रिया

यदि तपाईंको एजेन्ट प्रयोगकर्ता इन्टरफेसमा समावेश गरिएको छ भने, तपाईं प्रत्यक्ष प्रयोगकर्ता प्रतिक्रियाहरू (जस्तै च्याट UI मा थम्स-अप/डाउन) रेकर्ड गर्न सक्नुहुन्छ।


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

प्रयोगकर्ताको प्रतिक्रिया त्यसपछि Langfuse मा समेटिन्छ:

![प्रयोगकर्ताको प्रतिक्रिया Langfuse मा समेटिँदैछ](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. स्वचालित LLM-as-a-Judge स्कोरिङ

LLM-as-a-Judge अर्को तरिका हो जसले तपाईंको एजेन्टको उत्पादनलाई स्वचालित रूपमा मूल्याङ्कन गर्न मद्दत गर्दछ। तपाईं उत्पादनको शुद्धता, विषाक्तता, शैली, वा तपाईंलाई चासो लाग्ने कुनै पनि मापदण्डको मूल्याङ्कन गर्न छुट्टै LLM कल सेटअप गर्न सक्नुहुन्छ।

**कार्यप्रवाह**:
1. तपाईंले एउटा **मूल्याङ्कन टेम्प्लेट** परिभाषित गर्नुहुन्छ, जस्तै, "पाठ विषाक्त छ कि छैन जाँच गर्नुहोस्।"
2. तपाईंले एउटा मोडेल सेट गर्नुहुन्छ जुन न्यायाधीश-मोडेलको रूपमा प्रयोग हुन्छ; यस अवस्थामा `gpt-4o-mini` Azure मार्फत सोधिएको।
2. प्रत्येक पटक तपाईंको एजेन्टले उत्पादन सिर्जना गर्दा, तपाईं त्यो उत्पादनलाई टेम्प्लेटसहित "न्यायाधीश" LLM मा पठाउनुहुन्छ।
3. न्यायाधीश LLM ले रेटिङ वा लेबलको साथ प्रतिक्रिया दिन्छ, जुन तपाईंले आफ्नो अवलोकन उपकरणमा लग गर्नुहुन्छ।

Langfuse बाट उदाहरण:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

तपाईं यो उदाहरणको उत्तरलाई "हानिकारक छैन" भनेर मूल्याङ्कन गर्न सक्नुहुन्छ।

![LLM-को-रूपमा-न्यायाधीश मूल्याङ्कन स्कोर](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### ५. अवलोकनीयता मेट्रिक्सको अवलोकन

यी सबै मेट्रिक्सहरू ड्यासबोर्डहरूमा सँगै दृश्यात्मक रूपमा हेर्न सकिन्छ। यसले तपाईंलाई धेरै सत्रहरूमा तपाईंको एजेन्ट कसरी प्रदर्शन गरिरहेको छ भनेर छिटो हेर्न र समयसँगै गुणस्तर मेट्रिक्स ट्र्याक गर्न मद्दत गर्दछ।

![अवलोकनीयता मेट्रिक्सको अवलोकन](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## अफलाइन मूल्याङ्कन

अनलाइन मूल्याङ्कन प्रत्यक्ष प्रतिक्रिया प्राप्त गर्न महत्त्वपूर्ण छ, तर तपाईंलाई **अफलाइन मूल्याङ्कन** पनि आवश्यक छ—विकास अघि वा विकासको क्रममा व्यवस्थित जाँच। यसले उत्पादनमा परिवर्तन लागू गर्नु अघि गुणस्तर र विश्वसनीयता कायम राख्न मद्दत गर्दछ।


### डाटासेट मूल्यांकन

अफलाइन मूल्यांकनमा, सामान्यत:
1. तपाईंसँग एउटा बेंचमार्क डाटासेट हुन्छ (प्रम्ट र अपेक्षित नतिजा जोडीहरूसहित)
2. उक्त डाटासेटमा आफ्नो एजेन्ट चलाउनुहोस्
3. नतिजाहरूलाई अपेक्षित नतिजासँग तुलना गर्नुहोस् वा थप स्कोरिङ मेकानिजम प्रयोग गर्नुहोस्

तल, हामी [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset) सँग यो विधि प्रदर्शन गर्छौं, जसमा प्रश्नहरू र अपेक्षित उत्तरहरू समावेश छन्।


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

अर्को, हामी रनहरू ट्र्याक गर्नको लागि Langfuse मा एउटा डाटासेट इकाई सिर्जना गर्छौं। त्यसपछि, हामी डाटासेटबाट प्रत्येक वस्तु प्रणालीमा थप्छौं।


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![डाटासेट वस्तुहरू Langfuse मा](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### डेटासेटमा एजेन्ट चलाउने

पहिले, हामी एउटा साधारण Autogen एजेन्ट तयार गर्छौं जसले Azure OpenAI मोडेलहरू प्रयोग गरेर प्रश्नहरूको उत्तर दिन्छ।


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

त्यसपछि, हामी सहायक कार्य `my_agent()` परिभाषित गर्छौं।


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

अन्ततः, हामी प्रत्येक डाटासेट आइटममा लुप गर्छौं, एजेन्ट चलाउँछौं, र ट्रेसलाई डाटासेट आइटमसँग जोड्छौं। आवश्यक परेमा, हामी छिटो मूल्याङ्कन स्कोर पनि संलग्न गर्न सक्छौं।


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

महत्त्वपूर्ण नियमहरू:  
1. फरक फरक एजेन्ट कन्फिगरेसनहरूसँग यो प्रक्रिया दोहोर्याउन सकिन्छ, जस्तै:  
   - मोडेलहरू (gpt-4o-mini, gpt-4.1, आदि)  
   - प्रम्प्टहरू  
   - उपकरणहरू (सर्च बनाम बिना सर्च)  
   - एजेन्टको जटिलता (बहु-एजेन्ट बनाम एकल-एजेन्ट)  

2. त्यसपछि तिनीहरूलाई Langfuse मा साइड-बाइ-साइड तुलना गर्न सकिन्छ।  

यस उदाहरणमा, मैले २५ वटा डेटासेट प्रश्नहरूमा एजेन्टलाई ३ पटक चलाएँ। प्रत्येक रनका लागि, मैले फरक Azure OpenAI मोडेल प्रयोग गरें। तपाईं देख्न सक्नुहुन्छ कि ठूला मोडेल प्रयोग गर्दा सही उत्तर दिने प्रश्नहरूको संख्या बढ्छ (जसरी अपेक्षा गरिएको थियो)।  

`correct_answer` स्कोर [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) द्वारा सिर्जना गरिएको हो, जुन डेटासेटमा दिइएको नमूना उत्तरको आधारमा प्रश्नको शुद्धता न्याय गर्न सेटअप गरिएको छ।  

![डेटासेट रन अवलोकन](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)  
![डेटासेट रन तुलना](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)  



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
